In [22]:
%pwd

'h:\\AIdoctor\\MediJoy'

In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'h:\\AIdoctor\\MediJoy'

In [3]:
# Install langchain and langchain-community if not already installed
# %pip install langchain
# %pip install -U langchain-community

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# Ensure langchain is installed and up to date
# %pip install langchain

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [6]:
# Ensure pypdf is installed
# %pip install pypdf

# Extract data from the PDF file
extracted_data=load_pdf_file(data='Data/')

In [7]:
# extracted_data

In [8]:
# Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 39994


In [10]:
# text_chunks

In [11]:
# Ensure langchain is installed and up to date
# %pip install langchain

from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [13]:
# Ensure sentence-transformers is installed
# %pip install sentence-transformers

# Download the embeddings
embeddings = download_hugging_face_embeddings()

C:\Users\BiT-COUNTER\AppData\Local\Temp\ipykernel_22840\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')


In [18]:
# %pip uninstall -y protobuf google
# %pip install protobuf google


In [19]:
# Ensure protoc-gen-openapiv2 is installed
%pip install protoc-gen-openapiv2

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medijoy"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [23]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [24]:
#Load Existing index 

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("What is Acne?")

In [28]:
retrieved_docs

[Document(id='cd9e61a6-9d7d-4725-8a38-4bbd46c8f46d', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\bigdataset.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='cd47288e-605f-460f-b72d-c6fe1be0a067', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\bigdataset.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin— A drug that works by increasing the\nturnover (dea

In [ ]:
import openai
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv


# Set your correct OpenRouter API key
 
api_key = os.getenv("OPENROUTER_API_KEY")
# Configure OpenRouter API (Correct API base)
openai.api_base = "https://openrouter.ai/api/v1"

# Use ChatOpenAI with headers for OpenRouter
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",
    temperature=0.7,
    openai_api_key=api_key,  # OpenRouter-compatible API key
    openai_api_base="https://openrouter.ai/api/v1"  # Ensure correct API base
)

# Test the model
# response = llm.invoke("What is artificial intelligence?")
# print(response)


In [ ]:
#from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)


# from langchain_community.llms import Ollama

# llm = Ollama(model="llama2", temperature=0.4)


In [47]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks, especially for medical advisement and treatment suggestion. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [48]:
# Ensure retriever is defined
if 'docsearch' not in locals():
	from langchain_pinecone import PineconeVectorStore
	docsearch = PineconeVectorStore.from_existing_index(
		index_name=index_name,
		embedding=embeddings
	)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [44]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

 Acromegaly is a condition caused by an excess of growth hormone from the pituitary gland, leading to abnormal growth in bones and soft tissue. If left untreated, the disease does not worsen naturally. Gigantism, a similar condition that occurs in children, is also caused by an overproduction of growth hormone but from the pituitary gland during growth spurts. Growth hormone tests are used to confirm these conditions.


In [45]:
response = rag_chain.invoke({"input": "Do you know about ranula disease?"})
print(response["answer"])

 Based on the provided context, the text does not provide information about ranula disease. However, a ranula is a salivary gland cyst most commonly found on the floor of the mouth, and it's not related to the chronic granulomatous disease discussed in the text.


In [49]:
response = rag_chain.invoke({"input": "What do you know about head pain?"})
print(response["answer"])

 Head pain, often referred to as a headache, can result from various causes such as muscle tension, eyestrain, or issues with blood vessels in the head. Tension headaches are characterized by a steady, band-like pain that affects both sides of the head, and can be accompanied by symptoms like nausea, sensitivity to light and sound, and blurred vision. Migraines, another type of headache, can last longer and be more intense, with symptoms like throbbing pain, nausea, and potential sensitivity to light, sound, and odors. Pain is not caused by brain tissue itself, but can originate from structures like blood vessel walls, membranous brain coverings, scalp and neck muscles, and specific nerves of the face and head. Sinus inflammation can also cause headaches.
